# Pandas Tutorial Optional
Here I will show a few of the basic data visualization and manipulation tools available with Python Pandas. 

Here is a Pandas [cheatsheet](https://assets.datacamp.com/blog_assets/PandasPythonForDataScience.pdf).

I grabbed one year of household electricity data for demonstration purposes. **production** means solar production from a rooftop PV array during the given interval in watthours. **consumption** means the electricity consumed by the house in watthours. The goal will be to simply examine the data more closely. Sources:
* [Production](http://www.soda-is.com/eng/services/services_radiation_free_eng.php)
* [Consumption](http://www.smartgridaustralia.com.au/)

In [ ]:
import pandas as pd

## Read data from csv
The file 'data.csv' is in the same directory as this notebook, thus we don't need to specify the full path. I encourage you to open 'data.csv' from your [home base](http://localhost:8888/tree) to look at it.

`index_col=0` means that the first column in the CSV will be used as the index column. 

`parse_dates=[0]` means that the first column in the CSV will be converted from a string to a datetime object, which is much better for manipulation.

In [ ]:
try:
    df = pd.read_csv('data.csv', index_col=0, parse_dates=[0])
except FileNotFoundError:
    df = pd.read_csv('https://raw.githubusercontent.com/icanhazcodeplz/Jupyter-Notebook-Demo/master/data.csv', index_col=0, parse_dates=[0])

By default, notebooks display about 30 rows from the start and 30 rows from the end of the tabulated data.

In [ ]:
df

We can see that we have one year of data in 30 minute increments for the year 2013.

We can use `DataFrame.head(n)` and `DataFrame.tail(n)` to display the `n` rows at the beginning or end. Note that when you use `print()` you don't get the nice table output.

In [ ]:
print("Head of df")
print(df.head(5))
print("\nTail of df")
df.tail(5)

The Pandas developers are working on new 'styling' options. It's really cool!

Doing `df.style` prints ALL of the rows. This will take some time though because that's a lot to print! As a compromise, I'll take the first 100 rows and print them.

In [ ]:
rows_to_show = 100
df.head(rows_to_show).style

## Check for Duplicate Index Values
This is a good time to check for duplicate index values since we have a DateTime index.

`df.index.duplicated()` generates a True/False array, where True means that row has a duplicate index.

In [ ]:
df.index.duplicated()

Using df with brackets around a True/False array will return just the rows that are 'True'

In [ ]:
df[df.index.duplicated()]

Crap we have some dups! I'm too lazy to investigate why. I'll just drop them for now. 

`~` means "Not", so `df[~df.index.duplicated(keep='first')]` gives us all the rows that are not duplicates. I will reassign df to this.

In [ ]:
print("Size before dropping dubs =", df.size)
df = df[~df.index.duplicated(keep='first')]
print("Size after dropping dubs =", df.size)

# Prettier Tables!
There are tons of new ways to customize your tables that are documented [here](http://pandas.pydata.org/pandas-docs/stable/style.html). I'll show you a few.

### Color Gradient
You can use a color gradient with `df.style.background_gradient()`. The list of string cmap options are shown [here](http://matplotlib.org/users/colormaps.html).

In [ ]:
df.head(rows_to_show).style.background_gradient(cmap='Oranges') # Also try Reds, Blues, Greens, etc

### Bars

In [ ]:
df.head(rows_to_show).style.bar()

### Custom colors and a caption

Note that putting parenthesis around the expression allows you to put bullets on new lines so you can string together methods.

In [ ]:
(df.head(10).style
 .set_caption("Hi. I'm Caption, what's your name?")
 .set_properties(**{'background-color': 'black', 'color': 'lawngreen', 'border-color': 'red'})
)

# Plot the data
Jupyter supports two options for plotting with matplotlib, *inline* (static) and *notebook* (interactive).

The `%` sign means 'magic', which refers a IPython specific command. The full list is [here](https://ipython.org/ipython-doc/3/interactive/magics.html). You can also uncomment the code cell below to see all the options.

In [ ]:
# %magic # Uncomment me to see all the magic commands

In [ ]:
%matplotlib inline
df.plot()

Poopsicle. Something looks funny with the production data, how do you produce negative energy?! Let's use an interactive plot so we can look closely.

To zoom in, click the button with a box icon at the bottom of the plot, then select the area around the negative production values.

In [ ]:
# sometimes I have to run this cell twice to get it to work. Jupyter ain't perfect :/
%matplotlib notebook
df.plot(figsize=(8,5)) # figsize=(width, height) measured in inches

After zooming in, it looks like the weirdness only happens at some points of the day. August 27th is one of the bad days. To examine one day individually, use `df.loc[<yyyy-mm-dd>]`.

In [ ]:
df.loc['2013-08-27']

Looks like some missing data! -999 is a common value to use for a missing data point. Let's look at where production = -999. `df.describe()` prints column level statistics.

In [ ]:
df[df['production'] == -999].describe()

Ruhroh. 154 missing data points! And now the eternal question of how to handle missing data. Set to NaN? Set to 0? Throw it out completely? I'll set to 0 for now. Note that I had to use the `.loc` method when changing cells. Otherwise you get a warning. See http://pandas.pydata.org/pandas-docs/stable/indexing.html for all the ways to access and set values.

In [ ]:
df.loc[df.production == -999, 'production'] = 0
df.describe()

Be careful! This edits the variable `df` in the kernel, so all cells now use this updated `df`. Rerun the plotting cells above to see the difference.

## A few more helpful tips
I got these tips from a [datacamp tutorial](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.MfWKsbI)

 1. Put a ! in front of a line of code to run a shell command.
 1. Use `%who` to see all of the variables in your kernel.

In [ ]:
!echo "I'm a shell command!\n"
!ls

In [ ]:
%who